In [24]:
# -*- coding: utf-8 -*-
from ipywidgets import *
from IPython.display import display
import threading
import time
import smbus
import sys
import autodrive
reload(autodrive)
import LVEZ1
reload(LVEZ1)
import csv
import RPi.GPIO as GPIO
import Operate

LV = LVEZ1.Dist()
bus = smbus.SMBus(1)
D = autodrive.Car()
O = Operate.Operate()

In [32]:
# dlist1 = []
# f = open('distdata.csv', 'w')
# csvWriter = csv.writer(f)

dlist_front = []
dlist_frontL = []
dlist_frontR = []
dlist_behindR = []
dlist_back = []

class getdist(threading.Thread):
 
    def __init__(self):
         threading.Thread.__init__(self)

    def run(self):
        while True:
            dlist_front.append(LV.Measure(20))
            dlist_frontR.append(LV.Measure(21))
            dlist_frontL.append(LV.Measure(19))
            dlist_behindR.append(LV.Measure(26))
            dlist_back.append(LV.Measure(6))
            time.sleep(1)
            
 
th = getdist()
th.setDaemon(True)
th.start()

try:
    button1 = widgets.Button(description="Go ahead!")
    button2 = widgets.Button(description="Back!")
    button3 = widgets.Button(description="Stop!!")
    button4 = widgets.Button(description="Left Trun")
    button5 = widgets.Button(description="Right Trun")

    display(button1)
    display(button2)
    display(button3)
    display(button4)
    display(button5)

    def go_ahead(b):
        if handle.value == 0:
            D.forward(speed.value)

        elif handle.value > 0:
            D.right(speed.value,handle.value)

        elif handle.value < 0:
            D.left(speed.value,-1 * handle.value)

    def back(b):
        D.back(speed.value)

    def stop(b):
        D.brake()

    def l_turn(b):
        D.R_forward(speed.value)
        D.L_back(speed.value)

    def r_turn(b):
        D.L_forward(speed.value)
        D.R_back(speed.value)

    speed = IntSlider(description='Speed')
    display(speed)
    handle = IntSlider(min=-10, max=10, value=0,description='Handle_pitch')
    display(handle)
    button1.on_click(go_ahead)
    button2.on_click(back)
    button3.on_click(stop)
    button4.on_click(l_turn)
    button5.on_click(r_turn)

except KeyboardInterrupt:
    print(dlist_front)
#     csvWriter.writerow(dlist_front) 
#     writer = csv.writer(f, lineterminator='\n')
    GPIO.cleanup()
#     f.close()

In [41]:
f = open('distdata.csv', 'w')
csvWriter = csv.writer(f)
# print(dlist_frontL)
csvWriter.writerow(dlist_front) 
writer = csv.writer(f, lineterminator='\n')
csvWriter.writerow(dlist_frontR)
writer = csv.writer(f, lineterminator='\n')
csvWriter.writerow(dlist_frontL)
writer = csv.writer(f, lineterminator='\n')
csvWriter.writerow(dlist_behindR)
writer = csv.writer(f, lineterminator='\n')
csvWriter.writerow(dlist_back)
writer = csv.writer(f, lineterminator='\n')

f.close()